In [1]:
import datetime
import numpy
import netCDF4
from osgeo import gdal, osr
import ogr
import os
import pandas as pd

#example
#start = '2015-01-01'
#end = '2015-03-23'
#sd = dt.strptime(start,'%Y-%m-%d')
#ed = dt.strptime(end,'%Y-%m-%d')
#delta = ed - sd
#for i in range(delta.days+1):
#    dict[sd + td(days=i)] = <something>

reqStart = '1980-01-01'
reqEnd = '1981-12-31'

inputGeoCoordsFile = r'P:\projects\SILO\Wes\MNDB_geocoords.csv'
outputDir = r'P:\projects\SILO\Wes\testOut'
siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

varNameInSilo = 'et_morton_wet'
varForColName = 'pet'

#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'

In [2]:
#Read in GeoCoords
GeoCoordsVals = {}

geoCoordsDF = pd.read_csv(inputGeoCoordsFile)
#for index, row in geoCoordsDF.iterrows():
#    if not row[catGrpName] in GeoCoordsVals:
#        #print('Adding: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
#        GeoCoordsVals[row[catGrpName]] = row[catFileCatName]

allCats = geoCoordsDF[catGrpName].unique().tolist()
colNames = [varForColName + ' for ' + cat for cat in allCats]
print(len(allCats))

213


In [3]:

sd = datetime.datetime.strptime(reqStart,'%Y-%m-%d')
ed = datetime.datetime.strptime(reqEnd,'%Y-%m-%d')
##delta = ed - sd

sYear = sd.year
eYear = ed.year

allYears = []
for y in range(sYear, eYear + 1):
    if not y in allYears:
        allYears.append(y)

#date_today = datetime.now()
#days = pd.date_range(date_today, date_today + timedelta(7), freq='D')
date_rng = pd.date_range(sd, ed, freq='D')
#print(str(type(days[0])))
daysAsList = date_rng.tolist()
#mainDF = pd.DataFrame({'Date': days})
mainDF = pd.DataFrame(date_rng, columns=['tempDate'])
#mainDF = mainDF.set_index('Date')

#Getting date as index????
mainDF['Date'] = pd.to_datetime(mainDF['tempDate'])
mainDF = mainDF.set_index('Date')
mainDF.drop(['tempDate'], axis=1, inplace=True)


for col in colNames:
    mainDF[col] = ''


mainDF.head()
#print(mainDF)

,pet for SC #0,pet for SC #1,pet for SC #2,pet for SC #3,pet for SC #4,pet for SC #5,pet for SC #6,pet for SC #7,pet for SC #8,pet for SC #9,...,pet for SC #113,pet for SC #114,pet for SC #115,pet for SC #116,pet for SC #117,pet for SC #118,pet for SC #119,pet for SC #120,pet for SC #121,pet for SC #122
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,,,,,,,,,,,...,,,,,,,,,,
1980-01-02,,,,,,,,,,,...,,,,,,,,,,
1980-01-03,,,,,,,,,,,...,,,,,,,,,,
1980-01-04,,,,,,,,,,,...,,,,,,,,,,
1980-01-05,,,,,,,,,,,...,,,,,,,,,,


In [6]:
#load the data
print(allYears)

for thisYear in allYears:
    
    siloFile = os.path.join(siloNetCDFDir, str(thisYear) + '.' + varNameInSilo + '.nc')
    print('Processing: ' + siloFile)
    #print(type(thisYear))
    
    #Need the gdal version of silo to give us a few params, but will close of netCDF version pronto...
    siloDS = gdal.Open(siloFile)
    
    ##Need this to calc pixel size as well as work out order of y-axis
    netCDFData = netCDF4.Dataset(siloFile, 'r')

    #Grab the info we need to build GeoTransform object
    #Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array
    
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    first_lat = lats[:1]
    last_lat = lats[len(lats)-1:]
    #If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...
    
    first_lon = lons[:1]
    last_lon = lons[len(lons)-1:]
    
    colCount = siloDS.RasterXSize
    rowCount = siloDS.RasterYSize
    
    pixWidth = (last_lon - first_lon) / (siloDS.RasterXSize - 1)
    pixHeight = (last_lat - first_lat) / (siloDS.RasterYSize - 1)

    if first_lat < last_lat:
        #adjust for our use
        pixHeight *= -1
    
    
    siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
    siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre
    
    netCDFData.close()
    
    fakeSiloGT = [siloLeft, pixWidth, 0, siloTop, 0, pixHeight]
    
    
    
    #Jan 1
    siloDStart = datetime.datetime(year=thisYear, month=1, day=1)
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    for band in range( siloDS.RasterCount ):
        
        dateAdd = band
        dateNow = siloDStart + datetime.timedelta(days=dateAdd)
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        
        
        if not dateNow in daysAsList:
            continue

        #dateTimeStampKey = pd.Timestamp(dateNow, freq='D')#
        #dateTimeStampKey = datetime.datetime.strptime(dateForFileName,'%Y%m%d')
        dateTimeStampKey = str(dateNow.year) + '-' + str(dateNow.month).zfill(2) + '-' + str(dateNow.day).zfill(2)
        #print('Here: ' + str(dateTimeStampKey))
        #print('Here: ' + str(dateNow))
        
        band += 1
        #print("[ GETTING BAND ]: ", band)
        srcband = siloDS.GetRasterBand(band)
        
        if srcband is None:
            continue
            
        scaleFactor = 1.0
        
        sf = srcband.GetMetadataItem(varNameInSilo + '_scale_factor')
        nd = srcband.GetMetadataItem(varNameInSilo + '__FillValue')
        nd2 = srcband.GetNoDataValue()
        
        #if band == 1:
            #print('SRCBAND is : ' + str(type(srcband)))
            #print('ND: ' + str(nd) + ' and ND2: ' + str(nd2))
        
        #print(sf)
        
        scaleFactor = float(sf)
        rawNoData = float(nd)
        
        siloArray = srcband.ReadAsArray()
        #just make sure there's no gaps? Not necessary?
        #siloMask = numpy.ma.masked_where(siloArray == nd, siloArray)
        
        #print(dateForFileName + ': ' + str(siloMask[500,500] * scaleFactor))
        
        for thisCat in allCats:
            
            colName = varForColName + ' for ' + thisCat
            catTodaySum = 0.0
            catTodayCount = 0.0
            
            catDF = geoCoordsDF.loc[geoCoordsDF[catGrpName] == thisCat]
            
            for index, row in catDF.iterrows():
                
                longVal = float(row[longName])
                latVal = float(row[latName])
                countVal = int(row[countName])
                
                geoXcell = int((longVal - fakeSiloGT[0]) / fakeSiloGT[1]) #x pixel
                geoYcell = int((latVal - fakeSiloGT[3]) / fakeSiloGT[5]) #y pixel
                
                
                catTodaySum += (siloArray[geoYcell,geoXcell] * scaleFactor) * countVal
                catTodayCount += countVal
        
        
            #Calc average
            #avVal = 0
            if catTodayCount > 0:
                avVal = catTodaySum / catTodayCount
            #print('Today av: ' + str(avVal))
            #update mainDF
            mainDF.loc[dateTimeStampKey][colName] = avVal
        
#mainDF.head()
print('finished')

[1980, 1981]
Processing: P:\projects\SILO\Wes\mwet\1980.et_morton_wet.nc
Processing: P:\projects\SILO\Wes\mwet\1981.et_morton_wet.nc
finished


In [80]:
mainDF.head(20)

,pet for SC #0,pet for SC #1,pet for SC #2,pet for SC #3,pet for SC #4,pet for SC #5,pet for SC #6,pet for SC #7,pet for SC #8,pet for SC #9,...,pet for SC #113,pet for SC #114,pet for SC #115,pet for SC #116,pet for SC #117,pet for SC #118,pet for SC #119,pet for SC #120,pet for SC #121,pet for SC #122
Date,,,,,,,,,,,,,,,,,,,,,
1980-01-01,5.80395,5.45497,5.71017,5.80472,5.57516,5.65779,5.41401,5.78204,5.40071,5.53024,...,5.39291,5.68491,5.17656,5.54317,5.12403,5.43927,-11.382,5.30904,1.10539,5.90479
1980-01-02,7.24527,6.7963,7.14935,7.25682,7.01156,7.0905,6.84945,7.35118,6.75272,6.96085,...,5.42854,5.24124,5.34251,5.26779,5.6,5.25291,-12.3922,5.28505,-0.0457865,5.10581
1980-01-03,7.34765,7.41671,7.29844,7.29314,7.32594,7.25479,7.39135,7.32434,7.35227,7.29539,...,6.77097,6.75204,6.45004,6.88363,6.42403,6.64964,-10.5593,6.56586,2.19544,7.38152
1980-01-04,4.65889,5.32142,4.74785,4.54381,4.90699,4.8158,5.08136,4.40812,5.27714,4.99746,...,5.51623,6.04063,5.46834,5.77396,5.32403,5.76701,-11.1395,5.63548,1.37187,6.08643
1980-01-05,2.64365,2.8569,2.7,2.61609,2.70079,2.7,2.79169,2.60156,2.82674,2.78465,...,2.48038,2.65852,2.52912,2.54296,2.6,2.59533,-14.9495,2.55008,-2.34884,2.56554
1980-01-06,2.20078,2.10315,2.1527,2.11773,2.12626,2.10755,2.09297,2.1091,2.04404,2.08582,...,1.17289,1.44799,1.1,1.24359,1.1,1.16576,-15.6528,1.12685,-3.11814,1.69024
1980-01-07,2.65963,2.76917,2.64768,2.6338,2.69976,2.62613,2.75836,2.75269,2.6984,2.70823,...,1.27161,1.50837,0.966682,1.44723,1.02403,1.14481,-15.5624,1.06016,-3.16574,1.93291
1980-01-08,7.11249,6.96777,7.1159,7.09392,7.08948,7.10273,7.0449,7.09972,6.97249,7.09616,...,6.03006,5.94768,5.67692,6.05802,5.72403,5.83104,-11.4182,5.76586,1.31806,6.55655
1980-01-09,5.73866,4.88666,5.55396,5.70364,5.42277,5.44118,5.24961,5.76322,4.86534,5.26014,...,5.04196,5.32564,4.44386,5.38252,4.24805,5.02568,-11.3274,4.81052,1.46642,6.42368
